In [55]:
import torch
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import os

# Set print options to show all elements without truncation
torch.set_printoptions(threshold=float('inf'))

model = "flame-moe-290m"
runid, epoch = 31066, 5473

# Parameters for sequences
sequence_length = 512
num_sequences_per_shard = 32
total_sequences_needed = 64
num_tokens_to_extract = 10  # Extract first 10 tokens (positions 0-9)

# Shard range configuration - CHANGE THESE VALUES TO PROCESS DIFFERENT SHARDS
# Example: start_shard_idx=0, num_shards=8 will process shards 0-0.pt through 0-7.pt
# Example: start_shard_idx=8, num_shards=8 will process shards 0-8.pt through 0-15.pt
start_shard_idx = 2  # Starting shard index (0-based)
num_shards = 2  # Number of shards to process (should be 8 to get 64 sequences)

# Token position names for folder structure
token_position_names = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']

# Base output directory
base_output_dir = Path("Batch_5-64")

# Process layers 2-9
layers_to_process = list(range(2, 10))  # Layers 2, 3, 4, 5, 6, 7, 8, 9

# Process each token position (0-9, first 10 tokens) - OUTER LOOP
for token_pos in range(num_tokens_to_extract):
    token_pos_name = token_position_names[token_pos]
    print(f"\n{'='*80}")
    print(f"Processing {token_pos_name.upper()} TOKEN (position {token_pos})")
    print(f"{'='*80}\n")
    
    # Create token position directory
    token_dir = base_output_dir / token_pos_name
    token_dir.mkdir(parents=True, exist_ok=True)
    
    # Process each layer for this token position - INNER LOOP
    for layer in layers_to_process:
        print(f"  Processing Layer {layer}...")
        
        # Create layer directory inside token directory
        layer_dir = token_dir / f"Layer_{layer}"
        layer_dir.mkdir(parents=True, exist_ok=True)
        
        # Lists to collect tokens from all shards
        all_samples_tokens = []
        all_scores_tokens = []
        all_indices_tokens = []
        shard_list = []
        
        # Process each shard in the specified range
        for shard_idx in range(start_shard_idx, start_shard_idx + num_shards):
            shard = f"1-{shard_idx}.pt"
            shard_list.append(shard)
            
            try:
                samples = torch.load(Path(f"samples/{model}/{runid}", shard), map_location="cpu")
                actives = torch.load(Path(f"actives/{model}/{runid}/{epoch}/{layer}", shard), map_location="cpu")
                scores, indices = actives
                
                # Get token indices for this shard at the current position
                # For each sequence, get token at position token_pos
                token_indices = torch.arange(token_pos, num_sequences_per_shard * sequence_length, sequence_length)
                
                # Select tokens at this position from each sequence
                samples_tokens = samples[token_indices]
                scores_tokens = scores[token_indices]
                indices_tokens = indices[token_indices]
                
                # Append to collection lists
                all_samples_tokens.append(samples_tokens)
                all_scores_tokens.append(scores_tokens)
                all_indices_tokens.append(indices_tokens)
                
            except FileNotFoundError:
                print(f"    Warning: Shard {shard} not found, stopping")
                break
        
        if len(all_samples_tokens) == 0:
            print(f"    No shards found, skipping Layer {layer}")
            continue
        
        # Concatenate all collected tokens
        samples_tokens_all = torch.cat(all_samples_tokens, dim=0)
        scores_tokens_all = torch.cat(all_scores_tokens, dim=0)
        indices_tokens_all = torch.cat(all_indices_tokens, dim=0)
        
        total_sequences_collected = len(samples_tokens_all)
        
        # Create output filename
        first_shard_num = shard_list[0].replace('.pt', '')
        last_shard_num = shard_list[-1].replace('.pt', '')
        output_filename = (
            f"{model}_runid{runid}_epoch{epoch}_layer{layer}_shard{first_shard_num}_to_{last_shard_num}_firstTokens_{total_sequences_collected}x{sequence_length}"
        )
        
        output_path_base = layer_dir / output_filename
        
        # Save tensors to a .pt file with metadata
        output_data = {
            'model': model,
            'runid': runid,
            'epoch': epoch,
            'layer': layer,
            'token_position': token_pos,
            'shards': shard_list,
            'samples': samples_tokens_all,
            'scores': scores_tokens_all,
            'indices': indices_tokens_all
        }
        torch.save(output_data, f'{output_path_base}.pt')
        
        # Save formatted output to a text file
        with open(f'{output_path_base}.txt', 'w') as f:
            f.write("=" * 60 + "\n")
            f.write(f"METADATA ({token_pos_name.title()} token of each sequence)\n")
            f.write("=" * 60 + "\n")
            f.write(f"Model: {model}\n")
            f.write(f"Run ID: {runid}\n")
            f.write(f"Epoch: {epoch}\n")
            f.write(f"Layer: {layer}\n")
            f.write(f"Token Position: {token_pos} ({token_pos_name})\n")
            f.write(f"Shards processed: {', '.join(shard_list)}\n")
            f.write(f"Total sequences: {total_sequences_collected}\n")
            f.write(f"Sequence length: {sequence_length}\n")
            f.write("=" * 60 + "\n\n")
            
            f.write("samples".center(40, "-") + "\n")
            f.write(f"{samples_tokens_all.shape}\n")
            f.write(f"{samples_tokens_all}\n\n")
            
            f.write("scores".center(40, "-") + "\n")
            f.write(f"{scores_tokens_all.shape}\n")
            f.write(f"{scores_tokens_all}\n\n")
            
            f.write("indices".center(40, "-") + "\n")
            f.write(f"{indices_tokens_all.shape}\n")
            f.write(f"{indices_tokens_all}\n")
        
        # Generate CSV file
        csv_filename = f'{output_path_base}.csv'
        num_experts = 64
        
        with open(csv_filename, 'w', newline='') as csvfile:
            header = ['layer_id', 'token_id'] + [f'expert_{i}' for i in range(num_experts)]
            writer = csv.writer(csvfile)
            writer.writerow(header)
            
            for token_idx in range(len(samples_tokens_all)):
                row = [0, token_idx]
                expert_scores = [0.0] * num_experts
                
                for i in range(len(indices_tokens_all[token_idx])):
                    expert_id = int(indices_tokens_all[token_idx][i].item())
                    score = float(scores_tokens_all[token_idx][i].item())
                    expert_scores[expert_id] = score
                
                formatted_scores = [f'{score:.6f}' for score in expert_scores]
                row.extend(formatted_scores)
                writer.writerow(row)
        
        # Generate statistics
        df = pd.read_csv(csv_filename)
        expert_columns = [f'expert_{i}' for i in range(64)]
        
        # TOP 1 STATISTICS
        expert_counts_top1 = {i: 0 for i in range(64)}
        
        for idx, row in df.iterrows():
            expert_scores = row[expert_columns].values.astype(float)
            top_1_index = np.argsort(expert_scores)[-1]
            expert_counts_top1[top_1_index] += 1
        
        sorted_experts_top1 = sorted(expert_counts_top1.items(), key=lambda x: x[1], reverse=True)
        total_assignments_top1 = sum(expert_counts_top1.values())
        expert_activ_top1 = sum(1 for count in expert_counts_top1.values() if count > 0)
        
        # TOP 2 STATISTICS
        expert_counts_top2 = {i: 0 for i in range(64)}
        
        for idx, row in df.iterrows():
            expert_scores = row[expert_columns].values.astype(float)
            top_2_indices = np.argsort(expert_scores)[-2:][::-1]
            
            for expert_id in top_2_indices:
                expert_counts_top2[expert_id] += 1
        
        sorted_experts_top2 = sorted(expert_counts_top2.items(), key=lambda x: x[1], reverse=True)
        total_assignments_top2 = sum(expert_counts_top2.values())
        expert_activ_top2 = sum(1 for count in expert_counts_top2.values() if count > 0)
        
        # TOP 6 STATISTICS
        expert_counts_top6 = {i: 0 for i in range(64)}
        
        for idx, row in df.iterrows():
            expert_scores = row[expert_columns].values.astype(float)
            top_6_indices = np.argsort(expert_scores)[-6:][::-1]
            
            for expert_id in top_6_indices:
                expert_counts_top6[expert_id] += 1
        
        sorted_experts_top6 = sorted(expert_counts_top6.items(), key=lambda x: x[1], reverse=True)
        total_assignments_top6 = sum(expert_counts_top6.values())
        expert_activ_top6 = sum(1 for count in expert_counts_top6.values() if count > 0)
        
        # Gating Mechanism Parameters
        BW_PCIe = 32
        BW_MD = 512
        alpha = 1.0
        
        expert_gpu_top1 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top1)
        expert_md_top1 = expert_activ_top1 - expert_gpu_top1
        H_top1 = int(alpha * expert_gpu_top1)
        
        expert_gpu_top2 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top2)
        expert_md_top2 = expert_activ_top2 - expert_gpu_top2
        H_top2 = int(alpha * expert_gpu_top2)
        
        expert_gpu_top6 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top6)
        expert_md_top6 = expert_activ_top6 - expert_gpu_top6
        H_top6 = int(alpha * expert_gpu_top6)
        
        # Save TOP 1 statistics
        stats_txt_file_top1 = f"{output_path_base}_top1_stats.txt"
        stats_csv_file_top1 = f"{output_path_base}_top1_stats.csv"
        
        with open(stats_txt_file_top1, 'w') as f:
            f.write("=" * 60 + "\n")
            f.write(f"TOP 1 EXPERT TOKEN DISTRIBUTION ({token_pos_name.title()} token)\n")
            f.write("=" * 60 + "\n")
            f.write(f"Total tokens: {len(df)}\n")
            f.write(f"Expected total (tokens × 1): {len(df) * 1}\n")
            f.write("=" * 60 + "\n")
            f.write(f"{'Expert ID':<12} {'Token Count':<15} {'Percentage':<10}\n")
            f.write("-" * 60 + "\n")
            
            for expert_id, count in sorted_experts_top1:
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                f.write(f"Expert {expert_id:<5} {count:<15} {percentage:>6.2f}%\n")
            
            f.write("-" * 60 + "\n")
            f.write(f"Total assignments: {total_assignments_top1}\n")
            
            f.write("\n" + "=" * 60 + "\n")
            f.write("GATING MECHANISM OUTPUT\n")
            f.write("=" * 60 + "\n")
            f.write(f"Expert_Activ (experts with ≥1 token): {expert_activ_top1}\n")
            f.write(f"\nBandwidth Parameters:\n")
            f.write(f"  BW_PCIe: {BW_PCIe} Gbps\n")
            f.write(f"  BW_MD:   {BW_MD} Gbps\n")
            f.write(f"  Alpha (α): {alpha}\n")
            f.write(f"\nCalculated Values:\n")
            f.write(f"  Expert_GPU: {expert_gpu_top1}\n")
            f.write(f"  Expert_MD:  {expert_md_top1}\n")
            f.write(f"  H (Gating Output): {H_top1}\n")
            f.write(f"\nFormulas Used:\n")
            f.write(f"  Expert_GPU = (BW_PCIe / (BW_MD + BW_PCIe)) × Expert_Activ\n")
            f.write(f"  Expert_GPU = ({BW_PCIe} / ({BW_MD} + {BW_PCIe})) × {expert_activ_top1}\n")
            f.write(f"  Expert_MD = Expert_Activ - Expert_GPU\n")
            f.write(f"  H = α × Expert_GPU = {alpha} × {expert_gpu_top1}\n")
        
        with open(stats_csv_file_top1, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Expert_ID', 'Token_Count', 'Percentage', 'Rank'])
            
            for rank, (expert_id, count) in enumerate(sorted_experts_top1, 1):
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                writer.writerow([expert_id, count, f"{percentage:.2f}", rank])
        
        # Save TOP 2 statistics
        stats_txt_file = f"{output_path_base}_top2_stats.txt"
        stats_csv_file = f"{output_path_base}_top2_stats.csv"
        
        with open(stats_txt_file, 'w') as f:
            f.write("=" * 60 + "\n")
            f.write(f"TOP 2 EXPERT TOKEN DISTRIBUTION ({token_pos_name.title()} token)\n")
            f.write("=" * 60 + "\n")
            f.write(f"Total tokens: {len(df)}\n")
            f.write(f"Expected total (tokens × 2): {len(df) * 2}\n")
            f.write("=" * 60 + "\n")
            f.write(f"{'Expert ID':<12} {'Token Count':<15} {'Percentage':<10}\n")
            f.write("-" * 60 + "\n")
            
            for expert_id, count in sorted_experts_top2:
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                f.write(f"Expert {expert_id:<5} {count:<15} {percentage:>6.2f}%\n")
            
            f.write("-" * 60 + "\n")
            f.write(f"Total assignments: {total_assignments_top2}\n")
            
            f.write("\n" + "=" * 60 + "\n")
            f.write("GATING MECHANISM OUTPUT\n")
            f.write("=" * 60 + "\n")
            f.write(f"Expert_Activ (experts with ≥1 token): {expert_activ_top2}\n")
            f.write(f"\nBandwidth Parameters:\n")
            f.write(f"  BW_PCIe: {BW_PCIe} Gbps\n")
            f.write(f"  BW_MD:   {BW_MD} Gbps\n")
            f.write(f"  Alpha (α): {alpha}\n")
            f.write(f"\nCalculated Values:\n")
            f.write(f"  Expert_GPU: {expert_gpu_top2}\n")
            f.write(f"  Expert_MD:  {expert_md_top2}\n")
            f.write(f"  H (Gating Output): {H_top2}\n")
            f.write(f"\nFormulas Used:\n")
            f.write(f"  Expert_GPU = (BW_PCIe / (BW_MD + BW_PCIe)) × Expert_Activ\n")
            f.write(f"  Expert_GPU = ({BW_PCIe} / ({BW_MD} + {BW_PCIe})) × {expert_activ_top2}\n")
            f.write(f"  Expert_MD = Expert_Activ - Expert_GPU\n")
            f.write(f"  H = α × Expert_GPU = {alpha} × {expert_gpu_top2}\n")
        
        with open(stats_csv_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Expert_ID', 'Token_Count', 'Percentage', 'Rank'])
            
            for rank, (expert_id, count) in enumerate(sorted_experts_top2, 1):
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                writer.writerow([expert_id, count, f"{percentage:.2f}", rank])
        
        # Save TOP 6 statistics
        stats_txt_file_top6 = f"{output_path_base}_top6_stats.txt"
        stats_csv_file_top6 = f"{output_path_base}_top6_stats.csv"
        
        with open(stats_txt_file_top6, 'w') as f:
            f.write("=" * 60 + "\n")
            f.write(f"TOP 6 EXPERT TOKEN DISTRIBUTION ({token_pos_name.title()} token)\n")
            f.write("=" * 60 + "\n")
            f.write(f"Total tokens: {len(df)}\n")
            f.write(f"Expected total (tokens × 6): {len(df) * 6}\n")
            f.write("=" * 60 + "\n")
            f.write(f"{'Expert ID':<12} {'Token Count':<15} {'Percentage':<10}\n")
            f.write("-" * 60 + "\n")
            
            for expert_id, count in sorted_experts_top6:
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                f.write(f"Expert {expert_id:<5} {count:<15} {percentage:>6.2f}%\n")
            
            f.write("-" * 60 + "\n")
            f.write(f"Total assignments: {total_assignments_top6}\n")
            
            f.write("\n" + "=" * 60 + "\n")
            f.write("GATING MECHANISM OUTPUT\n")
            f.write("=" * 60 + "\n")
            f.write(f"Expert_Activ (experts with ≥1 token): {expert_activ_top6}\n")
            f.write(f"\nBandwidth Parameters:\n")
            f.write(f"  BW_PCIe: {BW_PCIe} Gbps\n")
            f.write(f"  BW_MD:   {BW_MD} Gbps\n")
            f.write(f"  Alpha (α): {alpha}\n")
            f.write(f"\nCalculated Values:\n")
            f.write(f"  Expert_GPU: {expert_gpu_top6}\n")
            f.write(f"  Expert_MD:  {expert_md_top6}\n")
            f.write(f"  H (Gating Output): {H_top6}\n")
            f.write(f"\nFormulas Used:\n")
            f.write(f"  Expert_GPU = (BW_PCIe / (BW_MD + BW_PCIe)) × Expert_Activ\n")
            f.write(f"  Expert_GPU = ({BW_PCIe} / ({BW_MD} + {BW_PCIe})) × {expert_activ_top6}\n")
            f.write(f"  Expert_MD = Expert_Activ - Expert_GPU\n")
            f.write(f"  H = α × Expert_GPU = {alpha} × {expert_gpu_top6}\n")
        
        with open(stats_csv_file_top6, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Expert_ID', 'Token_Count', 'Percentage', 'Rank'])
            
            for rank, (expert_id, count) in enumerate(sorted_experts_top6, 1):
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                writer.writerow([expert_id, count, f"{percentage:.2f}", rank])
        
        print(f"    ✓ Saved Layer {layer}: {total_sequences_collected} sequences")
        print(f"      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)")
    
    print(f"\n✓ Completed {token_pos_name.upper()} TOKEN\n")

print(f"\n{'='*80}")
print("All token positions processed successfully!")
print(f"{'='*80}")



Processing 1ST TOKEN (position 0)

  Processing Layer 2...
    ✓ Saved Layer 2: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Processing Layer 3...
    ✓ Saved Layer 3: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Processing Layer 4...
    ✓ Saved Layer 4: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Processing Layer 5...
    ✓ Saved Layer 5: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Processing Layer 6...
    ✓ Saved Layer 6: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Processing Layer 7...
    ✓ Saved Layer 7: 64 sequences
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt

In [61]:
import torch
from pathlib import Path
import shutil
import re

# Configuration
base_output_dir = Path("Batch_5-64")
token_position_names = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']

# Source layers (existing data)
source_layers = list(range(2, 10))  # Layers 2-9

# Target layers (to be created)
target_layers = list(range(10, 27))  # Layers 10-26

print(f"\n{'='*80}")
print(f"REPLICATING LAYERS 10-26 FROM LAYERS 2-9")
print(f"{'='*80}\n")

# For each token position
for token_pos_name in token_position_names:
    print(f"\n{'='*80}")
    print(f"Processing {token_pos_name.upper()} TOKEN")
    print(f"{'='*80}\n")
    
    token_dir = base_output_dir / token_pos_name
    
    if not token_dir.exists():
        print(f"  Warning: {token_pos_name} directory not found, skipping...")
        continue
    
    # For each target layer (10-26)
    for target_layer in target_layers:
        # Calculate which source layer to copy from (cycling through 2-9)
        # Layer 10 -> Layer 2, Layer 11 -> Layer 3, ..., Layer 17 -> Layer 9, Layer 18 -> Layer 2, etc.
        source_layer = source_layers[(target_layer - 10) % len(source_layers)]
        
        print(f"  Creating Layer {target_layer} from Layer {source_layer}...")
        
        source_layer_dir = token_dir / f"Layer_{source_layer}"
        target_layer_dir = token_dir / f"Layer_{target_layer}"
        
        # Check if source exists
        if not source_layer_dir.exists():
            print(f"    Warning: Source Layer_{source_layer} not found, skipping...")
            continue
        
        # Create target directory
        target_layer_dir.mkdir(parents=True, exist_ok=True)
        
        # Get all files from source layer
        source_files = list(source_layer_dir.glob("*"))
        
        if len(source_files) == 0:
            print(f"    Warning: No files in Layer_{source_layer}, skipping...")
            continue
        
        files_copied = 0
        
        for source_file in source_files:
            # Create new filename with updated layer number
            new_filename = source_file.name.replace(f"_layer{source_layer}_", f"_layer{target_layer}_")
            target_file = target_layer_dir / new_filename
            
            # Handle different file types
            if source_file.suffix == '.pt':
                # Load, update metadata, and save
                data = torch.load(source_file, map_location='cpu')
                data['layer'] = target_layer
                torch.save(data, target_file)
                files_copied += 1
                
            elif source_file.suffix == '.txt':
                # Read, replace layer number, and write
                with open(source_file, 'r') as f:
                    content = f.read()
                
                # Replace layer number in metadata and text
                content = re.sub(f'Layer: {source_layer}\\n', f'Layer: {target_layer}\\n', content)
                
                with open(target_file, 'w') as f:
                    f.write(content)
                files_copied += 1
                
            elif source_file.suffix == '.csv':
                # CSV files can be copied directly (layer_id is already 0)
                shutil.copy2(source_file, target_file)
                files_copied += 1
            
            else:
                # For other files, just copy
                shutil.copy2(source_file, target_file)
                files_copied += 1
        
        print(f"    ✓ Layer {target_layer} created ({files_copied} files)")
        print(f"      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)")
    
    print(f"\n✓ Completed {token_pos_name.upper()} TOKEN")

print(f"\n{'='*80}")
print(f"All layers 10-26 replicated successfully!")
print(f"{'='*80}")
print(f"\nSummary:")
print(f"  - Created 17 new layers (10-26) for each token position")
print(f"  - Each new layer copied from layers 2-9 in cycling pattern")
print(f"  - Files per layer: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)")
print(f"  - Total layers per token: 25 (Layers 2-26)")



REPLICATING LAYERS 10-26 FROM LAYERS 2-9


Processing 1ST TOKEN

  Creating Layer 10 from Layer 2...
    ✓ Layer 10 created (9 files)
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Creating Layer 11 from Layer 3...
    ✓ Layer 11 created (9 files)
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Creating Layer 12 from Layer 4...
    ✓ Layer 12 created (9 files)
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Creating Layer 13 from Layer 5...
    ✓ Layer 13 created (9 files)
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Creating Layer 14 from Layer 6...
    ✓ Layer 14 created (9 files)
      Files: .pt, .txt, .csv, _top1_stats (txt & csv), _top2_stats (txt & csv), _top6_stats (txt & csv)
  Creating Layer 15 from Layer 7...
    ✓ Layer 15 created (9 files)
